## landsat-dl
*01-11-2017 - Jasper Siebring*

Tool that allows you to search and generate download URLs for LANDSAT imagery. Originally based on Olivier Hagolle' [LANDSAT-DOWNLOAD](https://github.com/olivierhagolle/LANDSAT-Download) tool but with added support for collection 1 data and overall improvements across the board (also written in Python 3 instead of 2). Uses the USGS api to get any selected imagery' storage directories on the USGS servers which is needed to generate a valid download URL. Storage directories for pre-collection datasets are still manually assessed and can sadly change at any time. Collection 1' directories however are extracted directly from their API making it much more reliable (also much faster). 

**Features**:
- Query LANDSAT MSS/TM/7/8 imagery ([pre-collection and collection 1](https://landsat.usgs.gov/download-entire-collection-metadata))
- Filter using path/row/date/cloudcover/day/night
- Preview selected imagery (using preview image url found in metadata)  
- Automatically downloads (and updates if needed) metadata required for querying
- Generates download URLs for any selected LANDSAT imagery
- Directly download from generated URLs (requires logging in to the USGS website) using urllibs
- Checks validity of given dates and generated URLs
- Written in Python 3.5

**To be implemented**:
- Download using the requests library (current download functionality is limited and unnecessarily slow)
- Metadata query via some external database (preventing local downloading of large metadata files)


In [ ]:
##setup for Conda environment (not necessary but does provide easy package downloading)
#if conda is not installed (or not added to the command prompt) you will have to download the required packages elsewhere

#!conda create --name {env_name} python=3.5 jupyter
#!activate {env_name}
#!conda install usgs -c conda-forge
#!conda install pandas pillow requests dateutil datetime csv calendar io urllib

In [1]:
import re, os, datetime, sys, csv
from dateutil import parser
import requests
from usgs import api
import pandas as pd 
from calendar import monthrange 
from PIL import Image 
from io import BytesIO 
from usgs import USGSError

import urllib 
import urllib.request as depr_urllib2 

In [1]:
#login details for the USGS api and website
usgs_details = {'username' : 'your_username', 'password' : 'your_password'}

#     or enter your details in the given txt file and uncomment the following line
#usgs_details = get_usgs_details(txt_location)

api.login(usgs_details['username'], usgs_details['password'])

In [3]:
#downloads or updates preexisting metadata
def update_metadata(collection, end):
    #destdir = collection, might be list
    for destdir in collection: 
        if not os.path.exists(os.path.dirname(destdir)):
            print("Metadata (sub)folder is not found, creating..")
            os.makedirs(os.path.dirname(destdir))

        base_url = "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/"
        base = os.path.basename(destdir)

        url = os.path.join(base_url, base)
        if os.path.isfile(destdir):
            print("Metadata already exists, checking if up-to-date..")
            last_updated = requests.head(url)
            last_updated = parser.parse(last_updated.headers['Last-Modified']).date()
            if end > last_updated:
                print("Metadata not up-to-date, downloading..")
                download_file(url, destdir)
            elif end < last_updated:
                print("Metadata up-to-date for specified dates, cont..")
        else:    
            print ("Metadata not found, downloading {x} for the first time..".format(x = base))
            download_file(url, destdir)
            
#accepts both a list and single objects
def check_dates(dt):
    if (type(dt) == datetime.date or type(dt) == datetime.datetime):
        dt = [dt]
        
    boo_list = []
    for dt_obj in dt:
        boo_list.append((dt_obj.day == monthrange(dt_obj.year, dt_obj.month)[1] or dt_obj.day in range(1, monthrange(dt_obj.year, dt_obj.month)[1])))
    if len(boo_list) == 1:
        return(boo_list[0])
    return(boo_list)

def find_in_collection(destdir, sat_path, sat_row, sat, arc_col = 1, start=None, end=None, slc = False):
    
    #start lt5 collection 1 appears to be from 1984 onwards
    #pre-collection
    pre_l8 = pd.date_range(start= datetime.datetime(2013, 4, 11).date(), end=datetime.datetime(2017, 4, 30).date()) 
    pre_l7_slc_on = pd.date_range(start= datetime.datetime(1999, 1, 1).date(), end=datetime.datetime(2003, 5, 31).date()) 
    pre_l7_slc_off = pd.date_range(start= datetime.datetime(2003, 6, 1).date(), end=datetime.datetime(2017, 4, 30).date())
    pre_tm_1 = pd.date_range(start = datetime.datetime(1980, 1, 1).date(), end = datetime.datetime(1989, 12, 31).date())  
    pre_tm_2 = pd.date_range(start = datetime.datetime(1990, 1, 1).date(), end = datetime.datetime(1999, 12, 31).date())
    pre_tm_3 = pd.date_range(start = datetime.datetime(2000, 1, 1).date(), end = datetime.datetime(2009, 12, 31).date())
    pre_tm_4 = pd.date_range(start = datetime.datetime(2010, 1, 1).date(), end = datetime.datetime(2013, 12, 31).date())
    pre_mss1_1 = pd.date_range(start = datetime.datetime(1982, 1,1).date(), end = datetime.datetime(1997, 12, 31).date())
    pre_mss1_2 = pd.date_range(start = datetime.datetime(2012, 1,1).date(), end = datetime.datetime(2013, 12, 31).date())
    pre_mss2 = pd.date_range(start = datetime.datetime(1972, 1,1).date(), end = datetime.datetime(1983, 12, 31).date())

    #collection 1
    c1_l8 = pd.date_range(start= datetime.datetime(2013, 4, 11).date(), end=datetime.datetime.now().date()) 
    c1_l7 = pd.date_range(start= datetime.datetime(1999, 1, 1).date(), end=datetime.datetime.now().date()) 
    c1_tm = pd.date_range(start= datetime.datetime(1980, 1, 1).date(), end=datetime.datetime(2012, 12, 31).date())
    
    collection = []
    metadest = os.path.join(destdir, "metadata")
    
    if start == None:
        print("Startdate not given, reverting to launch of satellite {}".format(str(sat)))
        if sat == "LT5":
            if arc_col == 1:
                start = min(c1_tm).to_pydatetime().date()
            if arc_col == 0:
                start = min(pre_tm_1).to_pydatetime().date()
                
        if sat == "LE7":
            if arc_col == 1:
                start = min(c1_l7).to_pydatetime().date()
            if arc_col == 0:
                start = min(pre_l7_slc_on).to_pydatetime().date()
                
        if sat == "LC8":
            if arc_col == 1:
                start = min(c1_l8).to_pydatetime().date()
            if arc_col == 0:
                start = min(pre_l8).to_pydatetime().date()
    
    if end == None:
        print("Enddate not given, reverting to today (or end of satellite/collection)")
        if sat == "LT5":
            if arc_col == 1:
                end = max(c1_tm).to_pydatetime().date()
            if arc_col == 0:
                end = max(pre_tm_4).to_pydatetime().date()
        if sat == "LE7":
            if arc_col == 1:
                end = max(c1_l7).to_pydatetime().date()
            if arc_col == 0:
                end = max(pre_l7_slc_off).to_pydatetime().date() 
        if sat == "LC8":
            if arc_col == 1:
                end = datetime.datetime.now().date()
            if arc_col == 0:
                end = max(pre_l8).to_pydatetime().date()
    
    if not (check_dates(start) and check_dates(end)):
        print("One of the given dates doesn't exist, exiting..")
        sys.exit(1)
    
    ##################################
    ##pre-collection
    if arc_col == 0:
        subfolder = "pre_collection"
        if sat == "LC8": 
            if (start in pre_l8 or end in pre_l8):
                if not os.path.join(metadest, subfolder, "LANDSAT_8.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_8.csv"))
            else:
                print("Both dates not in satellite collection(s)..")
                

        if sat == "LE7":
            if (start in pre_l7_slc_on or end in pre_l7_slc_on):
                if not os.path.join(metadest, subfolder, "LANDSAT_ETM.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_ETM.csv"))
            
            elif (start in pre_l7_slc_off or end in pre_l7_slc_off):
                if not os.path.join(metadest, subfolder, "LANDSAT_ETM_SLC_OFF.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_ETM_SLC_OFF.csv"))
            else:
                print("Both dates not in satellite collection(s)..")
                #raise ValueError("Given dates are outside satellite collection..")

                    
        if sat == "LT5":
            if (start in pre_tm_1 or end in pre_tm_1):
                if not os.path.join(metadest, subfolder, "LANDSAT_TM-1980-1989.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_TM-1980-1989.csv"))
            
            elif (start in pre_tm_2 or end in pre_tm_2):
                if not os.path.join(metadest, subfolder, "LANDSAT_TM-1990-1999.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_TM-1990-1999.csv"))
            
            elif (start in pre_tm_3 or end in pre_tm_3):
                if not os.path.join(metadest, subfolder, "LANDSAT_TM-2000-2009.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_TM-2000-2009.csv"))
            
            elif (start in pre_tm_4 or end in pre_tm_4):
                if not os.path.join(metadest, subfolder, "LANDSAT_TM-2010-2012.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_TM-2010-2012.csv"))
            else:
                print("Both dates not in satellite collection(s)..")
        
        if sat == "MSS1":
            if (start in pre_mss1_1 or end in pre_mss1_1 or start in pre_mss1_2 or end in pre_mss1_2):
                if not os.path.join(metadest, subfolder, "LANDSAT_MSS1.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_MSS1.csv"))
            else:
                print("Both dates not in satellite collection(s)..")
        
        if sat == "MSS2":
            if (start in pre_mss2 or end in pre_mss2):
                if not os.path.join(metadest, subfolder, "LANDSAT_MSS2.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_MSS2.csv"))          
            else:
                print("Both dates not in satellite collection(s)..")
                
    #defaults to 1                            
    if arc_col == 1:
        subfolder = "collection_1"
        if sat == "LC8":
            if (start in c1_l8 or end in c1_l8):
                if not os.path.join(metadest, subfolder, "LANDSAT_8_C1.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_8_C1.csv"))
            else:
                print("Both dates not in satellite collection(s)..")
                
        if sat == "LE7":
            if (start in c1_l7 or end in c1_l7):
                if not os.path.join(metadest, subfolder, "LANDSAT_ETM_C1.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_ETM_C1.csv"))
            else:
                print("Both dates not in satellite collection(s)..")
                
        if sat == "LT5":
            if (start in c1_tm or end in c1_tm):
                if not os.path.join(metadest, subfolder, "LANDSAT_TM_C1.csv") in collection:
                    collection.append(os.path.join(metadest, subfolder, "LANDSAT_TM_C1.csv"))
            else:
                print("Both dates not in satellite collection(s)..")
        
    #updates required metadata files
    update_metadata(collection, end)

    files = []
    for col in collection:
        print("Searching for images in catalog {}".format(os.path.basename(col)))
        with open(col) as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if (int(row["path"]) == int(sat_path) and int(row["row"]) == int(sat_row) and row['dayOrNight'] == 'DAY'):
                    date_acq = parser.parse(row["acquisitionDate"]).date()
                    
                    if (start < date_acq < end):
                        files.append(row["sceneID"])
                        ##print(date_acq)
    print(str(len(files)) + " images have been found in " + str(len(collection)) + " collection(s)")
    return(files)

#bottleneck!
def generate_urls(sat_ids, destdir, sat, arc_col, usgs_details):
    urls = []
    
    #pre_collection is not on the API's db so the USGS_dir has to be found manually, these work though for the time being:
    if arc_col == 0:
        if sat == "LC8":
            usgs_dir = ['4923']
        if sat == "LE7":
            usgs_dir = ['3372', '3373']
        if sat == "LT5":
            usgs_dir = ['3119', '4345']
    
    if arc_col == 1:
        urls = list(map(lambda satids, usgs : "https://earthexplorer.usgs.gov/download/{}/{}/STANDARD/EE".format(usgs, satids), sat_ids, usgs_get_dir(sat_ids, sat, usgs_details)))
        print(str(len(urls)) + " URLs have been generated, returning..")
        return(urls)

    for product_id in sat_ids:        
        #product_tgz = os.path.join(destdir, product_id + '.tgz')
        #product_dest = os.path.join(destdir, product_id)

        for usgs_path in usgs_dir: 
            url = "https://earthexplorer.usgs.gov/download/{}/{}/STANDARD/EE".format(usgs_path, product_id)
            if requests.get(url).status_code == 200:
                urls.append(url)
                
    if not len(urls) == len(sat_ids):
        print(str(len(urls)) + " URLs have been generated with possible duplicates..")
    else:
        print(str(len(urls)) + " URLs have been generated, returning..")
    return(urls)


def download_file(url, destdir):
    r = requests.get(url, stream = True)
    with open(destdir, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    return
    
def usgs_get_dir(ids, sat, usgs_details):
    retries = 3
    if sat == "LC8":
        ls_dataset = 'LANDSAT_8_C1'
    if sat == "LE7":
        ls_dataset = 'LANDSAT_ETM_C1' #check differences in SLC!
    if sat == "LT5":
        ls_dataset = 'LANDSAT_TM_C1'
    
    #won't catch other non-USGS error (yet!)
    for i in range(retries):    
        try:
            usgs_dirs = list(map(lambda x: x['metadataUrl'].split("/")[5], api.metadata(ls_dataset, node = "EE", entityids=ids)['data']))
            return(usgs_dirs)
        except USGSError as u:
            if u.args[0].find("API") != -1:
                print("Could not find API key, retrying..")
                usgs_login(usgs_details=usgs_details)
            else:
                print(repr(u))
                continue
        else:
            break
    
def usgs_login(usgs_details):
    api.login(usgs_details['username'], usgs_details['password'])
    
def manual_assessment(sat_ids, sat, usgs_details):
    break_return = sat_ids[:]
    retries = 3
    or_len = len(sat_ids)
    if sat == "LC8":
        dataset = 'LANDSAT_8_C1'
    elif sat == "LE7":
        dataset = 'LANDSAT_ETM_C1' #check differences in SLC!
    elif sat == "LT5":
        dataset = 'LANDSAT_TM_C1'
    else:
        #print(str(sat) + " is not found in API, exiting..")
        raise NotImplementedError("The specified satellite has not been found")

    
    for i in range(retries):    
        try:
            previews = list(map(lambda x: x['browseUrl'], api.metadata(dataset=dataset, node = "EE", entityids=sat_ids)['data']))
            for i in range(len(previews)):
                response = requests.get(previews[i])
                img = Image.open(BytesIO(response.content))
                img.show()
                var = ""
                while len(var) == 0:
                    var = input((str(i + 1) + "/" + str(len(previews)) + " Keep this image? y/n  ")).lower()
                    if (var == "y" or var == "yes"):
                        continue
                    elif (var == "n" or var == "no"):
                        sat_ids[i] = None
                    elif (var == "q" or var == "quit" or var == "quit()"):
                        print("Quitting, returning original input")
                        return(break_return)
                    else:
                        continue
                img.close()

            sat_ids = list(filter(lambda x: bool(x), sat_ids))
            print("Returning " + str(len(sat_ids)) + "/" + str(or_len) + " ids")
            return(sat_ids)

        except USGSError as u:
            if u.args[0].find("API") != -1:    
                print("Could not find API key, retrying..")
                usgs_login(usgs_details=usgs_details)
            else:
                print(repr(u))
                continue
        else:
            break
            

def downloader(urls, destdir, usgs_details):
    for i in range(len(urls)):
        #SLOW, use IO/requests!
        product_id = urls[i].split("/")[5]
        product_tgz = os.path.join(destdir, product_id + '.tgz')
        if os.path.isfile(product_tgz):
            print(product_id + " is already found, cont..")
            continue
        
        product_dest = os.path.join(destdir, product_id)
        print("Downloading..  " + str(i + 1) + "/" + str(len(urls)))

        cookies = depr_urllib2.HTTPCookieProcessor()
        opener = depr_urllib2.build_opener(cookies)
        depr_urllib2.install_opener(opener)

        data = str(depr_urllib2.urlopen("https://ers.cr.usgs.gov").read())
        m = re.search(r'<input .*?name="csrf_token".*?value="(.*?)"', data)
        if m:
            token = m.group(1)
        else :
            print("Error : CSRF_Token not found")
            sys.exit(-3)

        params = urllib.parse.urlencode(dict(username=usgs_details['username'],password= usgs_details['password'], csrf_token=token)).encode("utf-8")
        request = urllib.request.Request("https://ers.cr.usgs.gov/login", params, headers={})
        f = depr_urllib2.urlopen(request)

        data = f.read()
        f.close()
        # if data.find('You must sign in as a registered user to download data or place orders for USGS EROS products')>0 :
        #     print("Authentification failed")
        #     sys.exit(-1)

        try:
            req = depr_urllib2.urlopen(urls[i])
            downloaded = 0
            CHUNK = 1024 * 1024 *64
            with open(product_tgz, 'wb') as fp:
                while True:
                    chunk = req.read(CHUNK)
                    downloaded += len(chunk)

                    if not chunk: break
                    fp.write(chunk)

        except:
            pass
        
        print(product_id + " has been downloaded to: " + destdir)

def get_usgs_details(txt):
    usgs_details = {}
    with open(txt) as file:
        for line in file:
            try:
                a, b = line.split()
                usgs_details['username'] = a
                usgs_details['password'] = b
            except:
                pass
    if not usgs_details['password']:
        raise ValueError("error with .txt file")        
    return(usgs_details)


In [14]:
#example of usage follows:

destdir = "M:/My Documents/git/landsat-dl/ls_imagery/"  
sat_path = '198'
sat_row = '23'
sat = 'LC8' #LC8, LT5 or LE7 
arc_col = 1 #or 0 without support for manual assessment   

#if left blank, will default to start/end of specified satellite
start = datetime.date(2014, 1, 1)
end = datetime.date(2014, 12, 1)

sat_ids = find_in_collection(start = start, end = end, sat_path = sat_path, sat_row=sat_row, sat = sat, destdir=destdir, arc_col=arc_col)
#sat_ids = manual_assessment(sat_ids, sat, usgs_details) #uses the default image handler 
urls = generate_urls(sat_ids, destdir=destdir, sat = sat, arc_col=arc_col, usgs_details= usgs_details)
#downloader(urls, destdir, usgs_details)

Metadata already exists, checking if up-to-date..
Metadata up-to-date for specified dates, cont..
Searching for images in catalog LANDSAT_8_C1.csv
21 images have been found in 1 collection(s)
21 URLs have been generated, returning..


Other databases:

http://lsiexplorer.cr.usgs.gov/
CWIC_LSI_EXPLORER_CATALOG_NODE = "CWIC"

http://earthexplorer.usgs.gov/
EARTH_EXPLORER_CATALOG_NODE = "EE"

http://hddsexplorer.usgs.gov/
HDDS_EXPLORER_CATALOG_NODE = "HDDS"

http://lpvsexplorer.cr.usgs.gov/
LPCS_EXPLORER_CATALOG_NODE = "LPCS"